In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
def dataset_to_numpy(dataset):
    images = []
    labels = []
    for image, label in dataset:
        images.append(image.numpy())
        labels.append(label.numpy())
    return np.array(images), np.array(labels)

In [3]:
def createTransferableModel(base_model,numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate):
    model = keras.Sequential()
    model.add(base_model)
    model.add(keras.layers.Flatten())
    
    for layer in range(numberOfFCLayers):
        if layer == numberOfFCLayers - 1:
            model.add(keras.layers.Dense(1,activation='sigmoid'))
        else:
            model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
            model.add(keras.layers.Dropout(dropoutRate))
            
    adamOptimizer = keras.optimizers.legacy.Adam(learning_rate=adamLearningRate)       
    model.compile(optimizer="adam",loss='binary_crossentropy', metrics=f1_score)
    return model

In [4]:
def main():
    train = basicUtil.loadData("training")
    dev,test = basicUtil.loadData("testing",.5)

    train_images, train_labels = dataset_to_numpy(train)

    base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_images[0].shape)
    base_model.trainable = False

    train = preprocess_input(train)
    dev = preprocess_input(dev)
    test = preprocess_input(test)

    possibleLayers = basicUtil.createRangeFromMidpoint(10,20)
    possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(10,20)
    possibleEpochs = basicUtil.createRangeFromMidpoint(50,100)

    dropoutCriticalPoints = (0,1)
    adamLearningRateCriticalPoints = (1e-4,1e-2)
    L2CriticalPoints = (1e-2,1e3) 

    n_FCLayers = []
    n_NeuronsPerFCLayers = []
    n_Epochs = []
    dropoutRates = []
    adamLearningRates = []
    L2Rates = []
    trainScores = []
    devScores = []

    trial = 0
    bestDevScore = 0
    
    while trial < 100:
        numberOfFCLayers = choose(possibleLayers)
        numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)

        numberOfEpochs = choose(possibleEpochs)
        
        dropoutRate = generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
        adamLearningRate = generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
        L2Rate = generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
        model = createTransferableModel(base_model,numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate)
        
        model.fit(train,epochs=numberOfEpochs,verbose=0)

        model_path = f'../Models/VGG16Trials/vgg16_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        if model_size < 40:
            print()
            print('trainScore')
            trainScore = model.evaluate(train)[1]
            print('devScore')
            devScore = model.evaluate(dev)[1]

            if (devScore > 0.91) and (devScore > bestDevScore):
                testScore = model.evaluate(test)[1]
                model_path = f'../Models/best_fd_model_.h5'
                model.save(model_path)
                bestModelParams = {
                    'n_FCLayers' : int(numberOfFCLayers),
                    'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                    'n_Epochs' : int(numberOfEpochs),
                    'dropoutRate' : dropoutRate,
                    'adamLearningRates' : adamLearningRate,
                    'L2Rates' : L2Rate,
                    'modelSize' : model_size,
                    'trainScore': trainScore,
                    'devScore': devScore,
                    'testScore': testScore
                }
                with open('../Models/best_vgg16_model_params.json', 'w') as f:
                    json.dump(bestModelParams, f)
                bestDevScore = devScore       

            n_FCLayers.append(numberOfFCLayers)
            n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
            n_Epochs.append(numberOfEpochs)
            
            adamLearningRates.append(adamLearningRate)
            dropoutRates.append(dropoutRate)
            L2Rates.append(L2Rate)
            trainScores.append(trainScore)
            devScores.append(devScore)
            
            print('concluding trial ',trial)
            trial += 1
        else:
            print(f'redoing trial {trial}. Model was {model_size}MB.')
            failedTrial = createTransferModelParametersDF([numberOfFCLayers],[numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                                  [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
            display(failedTrial)
        
            
        if (trial % 10 == 9): 
            modelParametersDF = createTransferModelParametersDF(n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                                                    dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
            modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
            display(modelParametersDF)
            
            top5 = modelParametersDF[0:5]
            possibleNumberOfFCLayers = basicUtil.getAdjustedRange(top5['n_FCLayers'])
            possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
        
            possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
            dropoutCriticalPoints = basicUtil.calculateCriticalPoints(top5['dropoutRate'])
            adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
            L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

            n_FCLayers = []
            n_NeuronsPerFCLayers = []
            n_Epochs = []
            dropoutRates = []
            adamLearningRates = []
            L2Rates = []
            trainScores = []
            devScores = []

            if bestDevScore > 0.91:
                trial = 101
                
    basicUtil.displayFinalResults('../Models/best_fd_model_params.json')
    

In [5]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-07-29 01:11:54.960724: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-29 01:11:54.960884: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-29 01:11:54.967060: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (75,) + inhomogeneous part.